## Для получения спарк сессии достаточно исполнить


In [21]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


## Настройка спарк сессии


In [22]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

## Информация о спарк сессии


In [23]:
spark

In [24]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [25]:
# http://files.grouplens.org/datasets/movielens/ml-100k.zip
rdd  = sc.textFile("/labs/laba01/ml-100k/u.data").map(lambda x: x.split("\t"))


In [26]:
rdd.take(5)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

In [27]:
rdd_film  = sc.textFile("/labs/laba01/ml-100k/u.item").map(lambda x: x.split("|"))

In [28]:
rdd_film.take(1) # 96

[['1',
  'Toy Story (1995)',
  '01-Jan-1995',
  '',
  'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)',
  '0',
  '0',
  '0',
  '1',
  '1',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0']]

## мой фильм  = 96

In [29]:
rdd_film.filter(lambda x: x[0] == '96').count() # проверяю, что в такой фильм только один

1

In [30]:
rdd_film.filter(lambda x: x[0] == '96').take(1) # вывожу инфу по фильму

[['96',
  'Terminator 2: Judgment Day (1991)',
  '01-Jan-1991',
  '',
  'http://us.imdb.com/M/title-exact?Terminator%202:%20Judgment%20Day%20(1991)',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '1',
  '0',
  '0']]

In [31]:
rdd_96 = rdd.filter(lambda x: x[1] == '96') # создаю датасет с оценками по моему фильму

In [32]:
# вывожу сколько оценок каждого типа по моему фильму
hist_film = []
for i in ['1', '2', '3', '4', '5']:
    cnt = rdd_96.filter(lambda x: x[2] == i).count()
    hist_film.append(cnt)
print(hist_film)

[6, 20, 43, 123, 103]


In [33]:
# вывожу сколько оценок каждого типа по всем фильмам
hist_all = []
for i in ['1', '2', '3', '4', '5']:
    cnt_all = rdd.filter(lambda x: x[2] == i).count()
    hist_all.append(cnt_all)
print(hist_all)

[6110, 11370, 27145, 34174, 21201]


In [34]:
data = {"hist_film": hist_film, "hist_all": hist_all} #собираю словарь для оформления ответа

In [35]:
# оформляю ответ
import json
with open('lab01.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [36]:
json.dumps(data) #проверяю ответ визуально

'{"hist_film": [6, 20, 43, 123, 103], "hist_all": [6110, 11370, 27145, 34174, 21201]}'

## После завершения работы правильно закрыть контекст

In [37]:
spark.stop()